## Loading Libraries

In [1]:
import pandas as pd

## Transforming Data

### Loading Dataset

In [2]:
df = pd.read_csv('../data/processed/train_sample_processed.csv')

In [3]:
df.head(5)

,Unnamed: 0,ReputationAtPostCreation,OwnerUndeletedAnswerCountAtPostTime,Title,BodyMarkdown,DaysTillPosting,RecognizedTags,UnrecognizedTags,OpenStatus
0,0,1,2,For Mongodb is it better to reference an objec...,I am building a corpus of indexed sentences in...,243,['mongodb'],4,open
1,1,192,24,How to insert schemalocation in a xml document...,i create a xml document with JAXP and search a...,122,['dom'],4,open
2,2,1,0,Too many lookup tables,What are the adverse effects of having too man...,0,"['sql-server', 'database-design']",3,open
3,3,4,1,What is this PHP code in VB.net,I am looking for the vb.net equivalent of this...,258,['vb.net'],4,too localized
4,4,334,14,Spring-Data mongodb querying multiple classes ...,"With Spring-Data, you can use the @Document an...",453,['mongodb'],4,open


### Encoding Token-like Data

### Encoding Tags